# パッケージインストール(Package install)

In [1]:
!pip install -U tf2onnx
!pip install onnxruntime

     |████████████████████████████████| 430 kB 5.2 MB/s 
     |████████████████████████████████| 12.3 MB 177 kB/s 
     |████████████████████████████████| 4.5 MB 5.1 MB/s 


# Saved Model ファイルダウンロード(Saved Model file download)

In [2]:
!wget -q -O object_detection_mobile_object_localizer_v1_1_default_1.tflite https://tfhub.dev/google/lite-model/object_detection/mobile_object_localizer_v1/1/default/1?lite-format=tflite

# tfliteからONNXへ変換(Convert from tflite to ONNX)

In [3]:
!python -m tf2onnx.convert --opset 11 \
    --tflite object_detection_mobile_object_localizer_v1_1_default_1.tflite \
    --output object_detection_mobile_object_localizer_v1_1_default_1.onnx

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-09-19 01:04:03,643 - INFO - Using tensorflow=2.6.0, onnx=1.10.1, tf2onnx=1.9.2/0f28b7
2021-09-19 01:04:03,643 - INFO - Using opset <onnx, 11>
2021-09-19 01:04:04,436 - WARNING - NMS node TFLite_Detection_PostProcess uses fast NMS. ONNX will approximate with standard NMS.
2021-09-19 01:04:04,519 - INFO - Optimizing ONNX model
2021-09-19 01:04:09,953 - INFO - After optimization: Cast -14 (18->4), Const -568 (1065->497), DequantizeLinear -12 (292->280), Identity -4 (4->0), QuantizeLinear -4 (114->110), Reshape -32 (46->14), Transpose -344 (353->9)
2021-09-19 01:04:10,037 - INFO - 
2021-09-19 01:04:10,037 - INFO - Successfully converted TensorFlow model object_detection_mobile_object_localizer_v1_1_default_1.tflite to ONNX
2021-09-19 01:04:10,037 - I

# ONNX最適化

In [4]:
!pip install onnx-simplifier

     |████████████████████████████████| 466 kB 5.1 MB/s 
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.3.6-py3-none-any.whl size=12873 sha256=4739190ce26c751b110df1f36bee9320dfa0b787668c7984ce29abb654ba61f3
  Stored in directory: /root/.cache/pip/wheels/0c/47/80/8eb21098e22c19d60b1c14021ee67442b4ad2d7991fdad46ba
Successfully built onnx-simplifier


In [5]:
!python -m onnxsim \
    object_detection_mobile_object_localizer_v1_1_default_1.onnx \
    object_detection_mobile_object_localizer_v1_1_default_1.onnx

Simplifying...
Checking 0/3...
Checking 1/3...
Checking 2/3...
Ok!


# 入力詳細・出力詳細確認(Check Input details & Output details)

In [6]:
import onnxruntime

In [7]:
onnx_session = onnxruntime.InferenceSession('object_detection_mobile_object_localizer_v1_1_default_1.onnx')
input_detail = onnx_session.get_inputs()
output_detail = onnx_session.get_outputs()

print(len(input_detail), len(output_detail))
print(input_detail[0])
print(output_detail[0])
print(output_detail[1])
print(output_detail[2])
print(output_detail[3])

1 4
NodeArg(name='normalized_input_image_tensor', type='tensor(uint8)', shape=[1, 192, 192, 3])
NodeArg(name='TFLite_Detection_PostProcess', type='tensor(float)', shape=[1, 100, 4])
NodeArg(name='TFLite_Detection_PostProcess:1', type='tensor(float)', shape=[1, 100])
NodeArg(name='TFLite_Detection_PostProcess:2', type='tensor(float)', shape=[1, 100])
NodeArg(name='TFLite_Detection_PostProcess:3', type='tensor(float)', shape=[1])
